In [50]:
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="example",
  database="oarebyue_0.3"
)
cursor = db.cursor()
cursor.execute("SELECT * FROM text")
texts = cursor.fetchall()

In [53]:
textDiscourse = []
totalTexts = len(texts)
counter = 1

for text in texts:
    cursor.execute(f'SELECT explicit_spelling FROM text_discourse where text_uuid = "{text[1]}" and explicit_spelling is not null and explicit_spelling not like "%…%" and explicit_spelling not like "%x%" and type = "word" order by word_on_tablet')
    discourse = cursor.fetchall()
    if len(discourse) > 50:
        cleanDiscourse = []
        for word in discourse:
            cleanDiscourse.append(word[0])
        textDiscourse.append({'uuid': text[1], 'words': ' '.join(cleanDiscourse)})
    print(f'{counter/totalTexts*100}% complete')
    counter = counter + 1

0.007954816641476414% complete
0.015909633282952827% complete
0.023864449924429244% complete
0.031819266565905654% complete
0.03977408320738207% complete
0.04772889984885849% complete
0.055683716490334906% complete
0.06363853313181131% complete
0.07159334977328773% complete
0.07954816641476414% complete
0.08750298305624056% complete
0.09545779969771698% complete
0.1034126163391934% complete
0.11136743298066981% complete
0.1193222496221462% complete
0.12727706626362262% complete
0.13523188290509902% complete
0.14318669954657545% complete
0.15114151618805186% complete
0.1590963328295283% complete
0.1670511494710047% complete
0.17500596611248112% complete
0.18296078275395752% complete
0.19091559939543395% complete
0.19887041603691033% complete
0.2068252326783868% complete
0.21478004931986316% complete
0.22273486596133962% complete
0.230689682602816% complete
0.2386444992442924% complete
0.24659931588576883% complete
0.25455413252724524% complete
0.2625089491687217% complete
0.270463765810

In [52]:
import pandas as pd

df = pd.DataFrame(textDiscourse)
df['words']
df.to_csv("textDiscourse.csv")

In [43]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical, pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [44]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    # newString = re.sub(r"'s\b","",newString)
    # # remove punctuations
    # newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    # print('Total Sequences: %d' % len(sequences))
    return sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [47]:
# preprocess the text
data_new = ''
model = Sequential()
totalTexts = len(textDiscourse)
counter = 1

for text in textDiscourse:
    data_new = text_cleaner(text['words'])
    # create sequences   
    sequences = create_seq(data_new)
    # create a character mapping index
    chars = sorted(list(set(data_new)))
    mapping = dict((c, i) for i, c in enumerate(chars))
    tempSequences = list()
    for line in sequences:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        tempSequences.append(encoded_seq)
    sequences = tempSequences
    # vocabulary size
    vocab = len(mapping)
    sequences = np.array(sequences)
    # create X and y
    X, y = sequences[:,:-1], sequences[:,-1]
    # one hot encode y
    y = to_categorical(y, num_classes=vocab)
    # create train and validation sets
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

    # print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)
    model.add(Embedding(vocab, 50, input_length=30, trainable=True))
    # model.add(LSTM(100, return_sequences=True))
    # model.add(LSTM(100))    
    model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dense(vocab, activation='softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
    # fit the model
    model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))
    print(f'{counter/totalTexts*100}% complete')
    counter = counter + 1
    

Epoch 1/100
16/16 - 5s - loss: 3.5143 - acc: 0.1786 - val_loss: 3.3711 - val_acc: 0.0893 - 5s/epoch - 326ms/step
Epoch 2/100
16/16 - 1s - loss: 3.1330 - acc: 0.1925 - val_loss: 3.2170 - val_acc: 0.0893 - 525ms/epoch - 33ms/step
Epoch 3/100
16/16 - 0s - loss: 2.9823 - acc: 0.1925 - val_loss: 3.1496 - val_acc: 0.0893 - 493ms/epoch - 31ms/step
Epoch 4/100
16/16 - 1s - loss: 2.8936 - acc: 0.1925 - val_loss: 3.1390 - val_acc: 0.0893 - 537ms/epoch - 34ms/step
Epoch 5/100
16/16 - 0s - loss: 2.8706 - acc: 0.1925 - val_loss: 3.1300 - val_acc: 0.0893 - 442ms/epoch - 28ms/step
Epoch 6/100
16/16 - 0s - loss: 2.8296 - acc: 0.2183 - val_loss: 3.0768 - val_acc: 0.1429 - 433ms/epoch - 27ms/step
Epoch 7/100
16/16 - 0s - loss: 2.7660 - acc: 0.2738 - val_loss: 3.0141 - val_acc: 0.1786 - 453ms/epoch - 28ms/step
Epoch 8/100
16/16 - 0s - loss: 2.6886 - acc: 0.2758 - val_loss: 3.0134 - val_acc: 0.1964 - 413ms/epoch - 26ms/step
Epoch 9/100
16/16 - 0s - loss: 2.5557 - acc: 0.3175 - val_loss: 2.8332 - val_acc: 

InvalidArgumentError: Graph execution error:

Detected at node 'sequential_8/embedding_15/embedding_lookup' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/topsecretmethlab/Library/Python/3.8/lib/python/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/topsecretmethlab/Library/Python/3.8/lib/python/site-packages/traitlets/config/application.py", line 845, in launch_instance
      app.start()
    File "/Users/topsecretmethlab/Library/Python/3.8/lib/python/site-packages/ipykernel/kernelapp.py", line 612, in start
      self.io_loop.start()
    File "/Users/topsecretmethlab/Library/Python/3.8/lib/python/site-packages/tornado/pla